## 生成随机数

In [38]:
import java.util.concurrent.ThreadLocalRandom
import breeze.linalg._
import scala.math._
import breeze.stats.distributions._
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.mllib.linalg.{Vectors,Matrix => sparkMatrix,DenseMatrix=> sparkDenseMatrix}

import java.util.concurrent.ThreadLocalRandom
import breeze.linalg._
import scala.math._
import breeze.stats.distributions._
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.mllib.linalg.{Vectors, Matrix=>sparkMatrix, DenseMatrix=>sparkDenseMatrix}


In [39]:
val N=sc.broadcast(1000)
val p=sc.broadcast(200)
val maxIter=sc.broadcast(6)
val Beta=sc.broadcast(DenseVector((1 to p.value).map(_.toDouble).map(s=> {
    if (s<=3) 2.0 else 0.0
}).toArray))
val Indeces=0 until N.value

N: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(68)
p: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(69)
maxIter: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(70)
Beta: org.apache.spark.broadcast.Broadcast[breeze.linalg.DenseVector[Double]] = Broadcast(71)
Indeces: scala.collection.immutable.Range = Range 0 until 1000


In [40]:
var norm_dist=new Gaussian(0,1)
val X = new DenseMatrix(rows = N.value, cols = p.value, norm_dist.sample(N.value*p.value).toArray)
val epsilon=(new DenseMatrix(rows=N.value,cols=1,norm_dist.sample(N.value).toArray)).toDenseVector
val y=X * Beta.value +epsilon
val inibetahat=inv(X.t*X)*X.t*y
var betahat=DenseVector.zeros[Double](p.value)

norm_dist: breeze.stats.distributions.Gaussian = Gaussian(0.0, 1.0)
X: breeze.linalg.DenseMatrix[Double] =
1.4451362742144553     -1.8789608963914892     ... (200 total)
0.9940427389257525     0.8827476082119183      ...
0.20388962227178903    0.7554817055295265      ...
-2.0862512363140526    -0.7917314304426422     ...
-0.8227252461840148    0.8371214924103401      ...
1.0382748782423563     0.7081604464392418      ...
-0.2531072036075817    1.1579114961191381      ...
-0.5559020120634642    -0.1408091172512501     ...
-0.28835435340644283   -0.22417732258390932    ...
0.8274859009571592     -2.181876728855322      ...
-1.091947925233896     0.30511273503780856     ...
0.6743966367795563     -0.7862772192955386     ...
0.06558858774183078    -2.0204012011181254     ...
...


## 第一问：串型程序

In [35]:
betahat(0 until p.value):=inibetahat(0 until p.value)  //这样写没什么不行的
val lam=sc.broadcast(0.85e-12)
var ii=0
var Diff=1.0
val eps=sc.broadcast(1e-5)
var Oldbeta=DenseVector.zeros[Double](p.value)
while(ii<maxIter.value && Diff>eps.value){
    Oldbeta:=betahat   //这样赋值简洁高效，但不写(0 until p.value)恐怕老师误判，所以咱们写copy，或就按之前那样写
    for(i<- 0 until p.value){  //p个分量逐个运算，典型的串型程序
        if(Oldbeta(i)!=0.0){
            if (math.abs(X(::,i).t*(y-X*Oldbeta))<lam.value) {
                betahat(i)=0.0
            }
            else {
                val tmp=DenseVector.zeros[Double](N.value)
                for(j<-0 until p.value){
                    if(j!=i) tmp+=X(::,j)*Oldbeta(j)
                }
                betahat(i)=X(::,i).t*(y-tmp)/(lam.value/math.abs(Oldbeta(i))+X(::,i).t*X(::,i))
            }
        }
    }
    Diff=sum((Oldbeta-betahat).map(math.abs(_)))
    println(Diff)
    ii+=1
}
println(betahat)

5.75248204378623
0.04067814927183755
0.0040434760893861554
4.0654399988726553E-4
4.031076940558265E-5
4.0425549674427685E-6
DenseVector(2.0150176520346417, 1.9521555322786428, 2.0905605690262408, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

lam: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(60)
ii: Int = 6
Diff: Double = 4.0425549674427685E-6
eps: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(61)
Oldbeta: breeze.linalg.DenseVector[Double] = DenseVector(2.015016236080513, 1.9521569343591099, 2.0905617935466125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...


## 第二问：并行程序

有办法解除+0和-0的

In [59]:
var norm_dist=new Gaussian(0,1)
val X = new DenseMatrix(rows = N.value, cols = p.value, norm_dist.sample(N.value*p.value).toArray)
val epsilon=(new DenseMatrix(rows=N.value,cols=1,norm_dist.sample(N.value).toArray)).toDenseVector
val y=X * Beta.value +epsilon
//先生成数据，再打标签
val inibetahat=inv(X.t*X)*X.t*y
var betahat=DenseVector.zeros[Double](p.value)
betahat:=inibetahat
val lam=sc.broadcast(0.85e-12)
var ii=0
var Diff=1.0
val eps=sc.broadcast(1e-5)
var Oldbeta=DenseVector.zeros[Double](p.value)
var x_array=(0 to p.value-1).map(s => (s,X(::,s),betahat(s))).toArray
//这个x的组建是关键，要的一定是X的列向量。这一定要在拿到题的时候通过维数等的分析确定到底是列向量组还是行向量组。维数分析很重要
var x=sc.parallelize(x_array)  //下面在不断地reassignment
var xs=DenseVector.zeros[Double](N.value)


while(ii<maxIter.value && Diff>eps.value){
    Oldbeta:=betahat //这样才能达到拷贝的效果
    xs:=x.map(s => {
        s._2*s._3
    }).reduce((x,y)=>x+y) //X*beta[m-1]
    val tmp=x.map(s => {
        if(Oldbeta(s._1)!=0.0){  //see whether the position is already been judged as 0. 
            if(math.abs(s._2.t*(y-xs))<lam.value){
                0.0
            }
            else {
                s._2.t*(y-(xs-s._2*Oldbeta(s._1)))/(lam.value/math.abs(Oldbeta(s._1))+s._2.t*s._2)
            }
        }else 0.0
    }).collect()
    betahat:=DenseVector(tmp)
    x_array=(0 to p.value-1).map(s => (s,X(::,s),betahat(s))).toArray
    x=sc.parallelize(x_array)
    Diff=sum((Oldbeta-betahat).map(math.abs(_)))
    ii+=1
}
println(betahat)

23/02/18 20:10:09 WARN TaskSetManager: Stage 143 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 20:10:09 WARN TaskSetManager: Stage 144 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 20:10:09 WARN TaskSetManager: Stage 145 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 20:10:10 WARN TaskSetManager: Stage 146 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 20:10:10 WARN TaskSetManager: Stage 147 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 20:10:10 WARN TaskSetManager: Stage 148 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 20:10:10 WARN TaskSetManager: Stage 149 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.

norm_dist: breeze.stats.distributions.Gaussian = Gaussian(0.0, 1.0)
X: breeze.linalg.DenseMatrix[Double] =
-0.775262611268067    -1.8533508772797553    ... (200 total)
-0.2601238543827679   0.6165901840218585     ...
0.20540843201261344   -0.6017389219399428    ...
-0.4170901733486124   -0.3423515982732988    ...
1.3277742053753199    1.8429706384003033     ...
6.498308169477537E-4  0.09898647972406638    ...
1.5865172851749385    0.6755009216949134     ...
1.4301981933404002    -0.6291794353299969    ...
-0.5691455983682173   1.2366438821703916     ...
0.44896585353781526   -0.6043190051313081    ...
-0.1641703061193133   -0.15395386207412234   ...
1.971139403745559     0.6428473700294344     ...
-0.9950936291675966   0.27725126972076913    ...
0.27049486474482914   -1.6...


In [65]:
var a=DenseVector(1.0,2.0,3.0)
var b=DenseVector(4.0,5.0,6.0)
b=copy(a)
a(0)=66.0
b

a: breeze.linalg.DenseVector[Double] = DenseVector(66.0, 2.0, 3.0)
b: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 2.0, 3.0)
b: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 2.0, 3.0)
res40: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 2.0, 3.0)
